In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from contants import openai_api_key

In [3]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-2024-08-06", api_key=openai_api_key)

# Define a team.
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Run the team and stream messages to the console.
stream = agent_team.run_stream(task="Write a beautiful poem 3-line about lake tangayika")

# Use asyncio.run(...) when running in a script.
await Console(stream)

# Save the state of the agent team.
team_state = await agent_team.save_state()


---------- TextMessage (user) ----------
Write a beautiful poem 3-line about lake tangayika
---------- TextMessage (assistant_agent) ----------
In Tanganyika's embrace, waters vast and deep,  
Whispers of age-old tales in gentle ripples sweep,  
Nature's azure mirror where earth and sky meet.  


In [4]:
await agent_team.reset()
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)


---------- TextMessage (user) ----------
What was the last line of the poem you wrote?


---------- TextMessage (assistant_agent) ----------
I'm sorry for any confusion, but I don't have the capability to write or remember personal content such as poems. However, I'd be happy to help you create a new poem or assist you with anything else you need! Let me know how I can assist you.


TaskResult(messages=[TextMessage(id='15b4d98f-90a4-4022-9fed-9749a0f540e5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 8, 15, 40, 55, 689998, tzinfo=datetime.timezone.utc), content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(id='a0fcdee8-c7b0-4b23-9fde-089e54691955', source='assistant_agent', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=52), metadata={}, created_at=datetime.datetime(2026, 2, 8, 15, 40, 56, 991211, tzinfo=datetime.timezone.utc), content="I'm sorry for any confusion, but I don't have the capability to write or remember personal content such as poems. However, I'd be happy to help you create a new poem or assist you with anything else you need! Let me know how I can assist you.", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [5]:
print(team_state)

# Load team state.
await agent_team.load_state(team_state)
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)


{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'assistant_agent': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a beautiful poem 3-line about lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': "In Tanganyika's embrace, waters vast and deep,  \nWhispers of age-old tales in gentle ripples sweep,  \nNature's azure mirror where earth and sky meet.  ", 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}, 'message_buffer': []}, 'RoundRobinGroupChatManager': {'type': 'RoundRobinManagerState', 'version': '1.0.0', 'message_thread': [{'id': '61db8529-d40a-4c61-b2b4-52a86d589bc1', 'source': 'user', 'models_usage': None, 'metadata': {}, 'created_at': '2026-02-08T15:40:30.673273Z', 'content': 'Write a beautiful poem 3-line about lake tangayika', 'type': 'TextMessage'}, {'id': 'b2b900b9-0597-4ce0-851a-ac6ce42a182d', 'sou

---------- TextMessage (assistant_agent) ----------
The last line of the poem I wrote was:  
"Nature's azure mirror where earth and sky meet."


TaskResult(messages=[TextMessage(id='986d9ea4-bdb7-4422-b0ce-522c4507cd9d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 8, 15, 41, 14, 47392, tzinfo=datetime.timezone.utc), content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(id='a82d3174-6f04-4419-a256-4d7c8cd20f84', source='assistant_agent', models_usage=RequestUsage(prompt_tokens=85, completion_tokens=22), metadata={}, created_at=datetime.datetime(2026, 2, 8, 15, 41, 14, 829771, tzinfo=datetime.timezone.utc), content='The last line of the poem I wrote was:  \n"Nature\'s azure mirror where earth and sky meet."', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [6]:
import json

## save state to disk

with open("team_state.json", "w") as f:
    json.dump(team_state, f)

## load state from disk
with open("team_state.json", "r") as f:
    team_state = json.load(f)

new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))
await new_agent_team.load_state(team_state)
stream = new_agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)
await model_client.close()


---------- TextMessage (user) ----------
What was the last line of the poem you wrote?
---------- TextMessage (assistant_agent) ----------
The last line of the poem was: "Nature's azure mirror where earth and sky meet."
